In [1]:
# import necessary modules for testing purpose
import pickle
import pandas as pd
import numpy as np

# Load data and save indices of columns
df = pd.read_excel("../data/dev.xlsx",converters={'MostRecentApp_No_Grp':str})

# Standard unpickle solution
This works now after we use a .py file to store corresponding funcs into pickles. Previously I tried to store them using jupyter notebook, which results in problems at point of unpickling.

In [2]:
# test over_cycle_pay_to_group function could be loaded and give right output
over_cycle_pay_to_group = pickle.load(open('../app/data/over_cycle_pay_to_group.pickle', 'rb'))
print(over_cycle_pay_to_group(-5000))

<=-3816


In [8]:
# test inq_main_to_group function could be loaded and give right output
inq_main_to_group = pickle.load(open('../app/data/func_feature2group_inq_main_to_group.pickle', 'rb'))
print(inq_main_to_group(0))
print(inq_main_to_group(1))
print(type(inq_main_to_group(1)))
print(inq_main_to_group(12))
print(inq_main_to_group(21))

0 - missing
1-9
<class 'numpy.ndarray'>
10-20
21 and +


In [4]:
# test group_to_woe function works properly
group_to_woe = pickle.load(open('../app/data/group_to_woe.pickle', 'rb'))

print(group_to_woe.__doc__)

# define the total bad and good volume
total_bad = np.sum(df.target == 1)
total_good = np.sum(df.target == 0)
group_to_woe(df, 'Rem2Month_flag_MonthsSince_Grp', total_bad, total_good)


    This function is relying on woe package and will calculate woe for a grouped variable
    :param df: dataframe containing grouped variables and target variable
    :param group_var: group variable to be used for calculating woe
    :param global_bt: total number of bads
    :param global_gt: total number of goods
    :param min_sample: min volume of sample in one bin
    :param alpha: min IV improvement needed for re-binning
    :return: panda series containing woe value calcualted from the original group info
    


0       -0.134188
1       -0.134188
2       -0.134188
3       -0.134188
4       -0.134188
           ...   
15095   -0.134188
15096   -0.134188
15097   -0.134188
15098   -0.134188
15099   -0.134188
Name: Rem2Month_flag_MonthsSince_Grp, Length: 15100, dtype: float64

In [5]:
# load in list of features
features = pickle.load(open('../app/data/features.pickle', 'rb'))
print(features)

# load in final model fit
model = pickle.load(open('../app/data/model.pickle', 'rb'))
print(model.__dict__)

['uc_woe', 'pmt_woe', 'cust_app_woe', 'land_woe', 'remain_pctg_woe', 'rem2_past_woe']
{'penalty': 'l2', 'dual': False, 'tol': 0.0001, 'C': 1.0, 'fit_intercept': True, 'intercept_scaling': 1, 'class_weight': None, 'random_state': None, 'solver': 'liblinear', 'max_iter': 100, 'multi_class': 'warn', 'verbose': 0, 'warm_start': False, 'n_jobs': None, 'l1_ratio': None, 'classes_': array([0, 1], dtype=int64), 'coef_': array([[0.38864868, 0.72627931, 0.69106253, 0.69807196, 0.83074297,
        0.57339894]]), 'intercept_': array([-2.26939282]), 'n_iter_': array([6])}


# Customized unpickler solution
This solution is not needed after adjustment at the point of serialization.
The adjustment is to use save_func_to_pickle.py to store the corresponding functions so the name space is correctly defined.

In [2]:
class MyCustomUnpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == "__main__":
            module = "model"
        return super().find_class(module, name)

In [3]:
with open('../app/data/inq_main_to_group.pickle', 'rb') as f:
    unpickler = MyCustomUnpickler(f)
    inq_main_to_group = unpickler.load()

# Print test result
print("Input 0: ", inq_main_to_group(0))
print("Input 1: ", inq_main_to_group(1))
print("Input 12: ", inq_main_to_group(12))
print("Input 21: ", inq_main_to_group(21))

Input 0:  0 - missing
Input 1:  1-9
Input 12:  10-20
Input 21:  21 and +


In [6]:
with open('../app/data/group_to_woe.pickle', 'rb') as f:
    unpickler = MyCustomUnpickler(f)
    group_to_woe = unpickler.load()

print(group_to_woe.__doc__)

# define the total bad and good volume
total_bad = np.sum(df.target == 1)
total_good = np.sum(df.target == 0)
group_to_woe(df, 'Rem2Month_flag_MonthsSince_Grp', total_bad, total_good)


    This function is relying on woe package and will calculate woe for a grouped variable
    :param df: dataframe containing grouped variables and target variable
    :param group_var: group variable to be used for calculating woe
    :param global_bt: total number of bads
    :param global_gt: total number of goods
    :param min_sample: min volume of sample in one bin
    :param alpha: min IV improvement needed for re-binning
    :return: panda series containing woe value calcualted from the original group info
    


0       -0.134188
1       -0.134188
2       -0.134188
3       -0.134188
4       -0.134188
           ...   
15095   -0.134188
15096   -0.134188
15097   -0.134188
15098   -0.134188
15099   -0.134188
Name: Rem2Month_flag_MonthsSince_Grp, Length: 15100, dtype: float64

In [9]:
# load in list of features
features = pickle.load(open('../app/data/features.pickle', 'rb'))
print(features)

['uc_woe', 'pmt_woe', 'cust_app_woe', 'land_woe', 'remain_pctg_woe', 'rem2_past_woe']


In [10]:
# load in final model fit
model = pickle.load(open('../app/data/model.pickle', 'rb'))
print(model.__dict__)

{'penalty': 'l2', 'dual': False, 'tol': 0.0001, 'C': 1.0, 'fit_intercept': True, 'intercept_scaling': 1, 'class_weight': None, 'random_state': None, 'solver': 'liblinear', 'max_iter': 100, 'multi_class': 'warn', 'verbose': 0, 'warm_start': False, 'n_jobs': None, 'l1_ratio': None, 'classes_': array([0, 1], dtype=int64), 'coef_': array([[0.38864868, 0.72627931, 0.69106253, 0.69807196, 0.83074297,
        0.57339894]]), 'intercept_': array([-2.26939282]), 'n_iter_': array([6])}
